In [ ]:
import zipfile

with zipfile.ZipFile("../input/generative-dog-images/all-dogs.zip","r") as zip_ref:
    zip_ref.extractall("/kaggle/temp/")
    
with zipfile.ZipFile("../input/generative-dog-images/Annotation.zip","r") as zip_ref:
    zip_ref.extractall("/kaggle/temp/")

In [ ]:
import os,gc
import numpy as np
from PIL import Image
import tensorflow as tf
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Flatten,Dense, Input, Embedding, Conv2D, MaxPooling2D, Reshape, BatchNormalization,\
concatenate, Conv2DTranspose, LeakyReLU
from tensorflow.keras.initializers import RandomNormal
import time
kernel_start = time.time()
LIMIT = 8

In [ ]:
ROOT = "/kaggle/temp/"
breeds = os.listdir(ROOT + "Annotation")
idxIn = 0; namesIn = []
imagesIn = np.zeros((25000,80,80,3))
# Extract data
for breed in breeds:
    for dog in os.listdir(ROOT+'Annotation/'+breed):
        try: img = Image.open(ROOT+'all-dogs/'+dog+'.jpg') 
        except: continue           
        ww,hh = img.size
        tree = ET.parse(ROOT+'Annotation/'+breed+'/'+dog)
        root = tree.getroot()
        objects = root.findall('object')
        for o in objects:
            bndbox = o.find('bndbox') 
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            w = np.min((xmax - xmin, ymax - ymin))
            # ADD PADDING TO CROPS
            EXTRA = w//8
            a1 = EXTRA; a2 = EXTRA; b1 = EXTRA; b2 = EXTRA
            a1 = np.min((a1,xmin)); a2 = np.min((a2,ww-xmin-w))
            b1 = np.min((b1,ymin)); b2 = np.min((b2,hh-ymin-w))
            img2 = img.crop((xmin-a1, ymin-b1, xmin+w+a2, ymin+w+b2))
            img2 = img2.resize((80,80), Image.ANTIALIAS)
            imagesIn[idxIn,:,:,:] = np.asarray(img2)
            namesIn.append(breed)                
            #if idxIn%1000==0: print(idxIn)
            idxIn += 1
                
#     idx = np.arange(idxIn)
#     np.random.shuffle(idx)
#     imageIN = imagesIn[idx,:,:,:]
#     namesIn = np.array(namesIn)[idx]


In [ ]:
x = np.random.randint(0,idxIn,25)
for k in range(5):
    plt.figure(figsize=(15,3))
    for j in range(5):
        plt.subplot(1,5,j+1)
        img = Image.fromarray( imagesIn[x[k*5+j],:,:,:].astype('uint8') )
        plt.axis('off')
        plt.title(namesIn[x[k*5+j]].split('-')[1],fontsize=11)
        plt.imshow(img)
    plt.show()

In [ ]:
# FUNCTION FOR DATA AUGMENTATION
def flip(x: tf.Tensor, y:tf.Tensor) -> (tf.Tensor,tf.Tensor):
    x = tf.image.random_flip_left_right(x)
    return (x,y)

# FUNCTION FOR DATA AUGMENTATION
def crop(x: tf.Tensor, y:tf.Tensor) -> (tf.Tensor,tf.Tensor):
    x = tf.image.random_crop(x,size=[64,64,3])
    return (x,y)

In [ ]:
BATCH_SIZE = 32
for i in range(len(namesIn)):
    namesIn[i] = namesIn[i].split('-')[1].lower()
label = LabelEncoder()
namesIn = label.fit_transform(namesIn)
namesIn = namesIn[:idxIn]
namesIn = namesIn.astype("int8")
imagesIn = (imagesIn[:idxIn,:,:,:]-127.5)/127.5
imagesIn = imagesIn.astype("float32") # 3935
# data pipeline
# ds = tf.data.Dataset.from_tensor_slices((imageIN,nameIN)).map(flip).map(crop).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# Z dim
MAPS = 128
noise_dim = 128

from tensorflow.keras import layers
from tensorflow.keras.initializers import RandomNormal
init = RandomNormal(mean=0.0, stddev=0.02)

def make_generator():
    seed = tf.keras.Input(shape=((noise_dim,)))
    label = tf.keras.Input(shape=((1,)))
    x = layers.Embedding(120, 120, input_length=1,name='emb')(label)
    x = layers.Flatten()(x)
    x = layers.concatenate([seed,x])
    x = layers.Dense(4*4*MAPS*8, use_bias=False)(x)
    x = layers.Reshape((4, 4, MAPS*8))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(MAPS*4, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(MAPS*2, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(MAPS, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False, activation='tanh')(x)

    model = tf.keras.Model(inputs=[seed,label], outputs=x)    
    return model

generator = make_generator()
generator.summary()
# non trainable param  = 1/2  param from BatchNormalization = 1/2 * sum (dim[4th] * 4)

In [ ]:
# Discriminator
def make_discriminator():
    image = tf.keras.Input(shape=((64,64,3)))
    label = tf.keras.Input(shape=((1,)))
    x = layers.Embedding(120, 64*64, input_length=1)(label)
    x = layers.Reshape((64,64,1))(x)
    x = layers.concatenate([image,x])
    
    x = layers.Conv2D(MAPS, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*2, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*4, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*8, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(121, activation='sigmoid')(x)
    x2 = layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=[image,label], outputs=[x,x2])
    return model

discriminator = make_discriminator()
discriminator.summary()

In [ ]:
#Optimizer
lr = tf.Variable(0.0002)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1 = 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1 = 0.5)

In [ ]:
DISPLAY_EVERY = 10

def display_images(model, test_input, labs):
    predictions = model([test_input,labs], training=False)
    fig = plt.figure(figsize=(16,4))
    for i in range(predictions.shape[0]):
        plt.subplot(2, 8, i+1)
        plt.imshow( (predictions[i, :, :, :]+1.)/2. )
        plt.axis('off')
    plt.show()
    
def generate_latent_points(latent_dim, n_samples):
    return tf.random.truncated_normal((n_samples,latent_dim))

def train(epochs):
    all_gl = np.array([]); all_dl = np.array([])
    
    for epoch in range(epochs):
        start = time.time()
        gl = []; dl = []
           
        idx = np.arange(idxIn)
        np.random.shuffle(idx)
        dataset = (tf.data.Dataset.from_tensor_slices((imagesIn[idx,:,:,:],namesIn[idx]))
            .map(flip).map(crop).batch(BATCH_SIZE,drop_remainder=True))
        
        # TRAIN ACGAN
        for i,image_batch in enumerate(dataset):
            gg,dd = train_step(image_batch,generator,discriminator,
                        generator_optimizer, discriminator_optimizer)
            gl.append(gg); dl.append(dd)
        all_gl = np.append(all_gl,np.array([gl]))
        all_dl = np.append(all_dl,np.array([dl]))
        
        # EXPONENTIALLY DECAY LEARNING RATES
        if epoch>180: learning_rate.assign(learning_rate*0.95)
        
        # DISPLAY PROGRESS
        if epoch%DISPLAY_EVERY==0:
            # PLOT EPOCH LOSS
            plt.figure(figsize=(16,2))
            plt.plot(np.arange(len(gl)),gl,label='Gen_loss')
            plt.plot(np.arange(len(dl)),dl,label='Disc_loss')
            plt.legend()
            plt.title('Epoch '+str(epoch)+' Loss')
            ymax = plt.ylim()[1]
            plt.show()
            
            # PLOT ALL TIME LOSS
            plt.figure(figsize=(16,2))
            plt.plot(np.arange(len(all_gl)),all_gl,label='Gen_loss')
            plt.plot(np.arange(len(all_dl)),all_dl,label='Disc_loss')
            plt.legend()
            plt.ylim((0,np.min([1.1*np.max(all_gl),2*ymax])))
            plt.title('All Time Loss')
            plt.show()

            # DISPLAY IMAGES FROM TRAIN PROGRESS
            seed = generate_latent_points(noise_dim, num_examples)
            labs = tf.cast(120*tf.random.uniform((num_examples,1)),tf.int8)
            display_images(generator, seed, labs)
            
            # PRINT STATS
            print('EPOCH',epoch,'took',np.round(time.time()-start,1),'sec')
            print('Gen_loss mean=',np.mean(gl),'std=',np.std(gl))
            print('Disc_loss mean=',np.mean(dl),'std=',np.std(dl))
            print('Learning rate = ',end='')
            tf.print(discriminator_optimizer.lr)
            
        x = gc.collect()
        tt = np.round( (time.time() - kernel_start)/60,1 )
        if tt > LIMIT*60: break
        
                  

In [ ]:
EPOCHS = 250
num_examples = 16

@ tf.function
def train_step(images,generator,discriminator,generator_optimizer,discriminator_optimizer):
        
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.4)
    bce2 = tf.keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0.4)
    noise = tf.random.normal((32,128)) # update noise_dim here
    labs = tf.cast(120*tf.random.uniform((32,)),tf.int32)
    
    # USE GRADIENT TAPE TO CALCULATE GRADIENTS
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:       
        generated_images = generator([noise,labs], training=True)
        real_cat, real_output = discriminator([images[0],images[1]], training=True)
        fake_cat, fake_output = discriminator([generated_images,labs], training=True)
    
        # GENERATOR LOSS 
        gen_loss = (tf.reduce_mean( (real_output - tf.reduce_mean(fake_output,0) + tf.ones_like(real_output))**2,0 )
        + tf.reduce_mean( (fake_output - tf.reduce_mean(real_output,0) - tf.ones_like(real_output))**2,0 ) )/2.
        
        # DISCRIMINATOR LOSS
        disc_loss = bce(tf.ones_like(real_output), real_output) + bce(tf.zeros_like(fake_output), fake_output)           
        real_cat2 = tf.one_hot(tf.cast(images[1],tf.int32),121,dtype=tf.int32)
        fake_cat2 = tf.one_hot(120*tf.ones((32,),tf.int32),121,dtype=tf.int32)
        disc_loss += bce2(real_cat2,real_cat) + bce2(fake_cat2,fake_cat) 
        
    # BACK PROPAGATE ERROR
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
       
    return gen_loss, disc_loss

    

In [ ]:
import warnings
warnings.filterwarnings("ignore")
train(EPOCHS)

In [ ]:
mse = tf.keras.losses.MeanSquaredError()

print('Display Random Dogs by Breed')
print()
for j in np.random.randint(0,120,25):
    # GENERATE DOGS
    seed = generate_latent_points(noise_dim, 10)
    labs = tf.cast( j*np.ones((10,1)), tf.int8)
    predictions = generator([seed,labs], training=False); d = 0   
    # GET BREED NAME    
    br = np.argwhere( namesIn==j ).flatten()
    bd = label.inverse_transform(np.array([j]))[0].capitalize()
    # CALCULATE VARIETY
    for k in range(4): d += mse(predictions[k,:,:,:],predictions[k+1,:,:,:]) 
    d = np.round( np.array(d),1 )
    if d<1.0: 
        print(bd,'had mode collapse. No display. (variety =',d,')')
        continue
    # DISPLAY DOGS
    print(bd,'REAL DOGS on top. FAKE DOGS on bottom. (variety =',d,')')
    plt.figure(figsize=(15,9))
    for i in range(5):
        plt.subplot(3,5,i+1)
        plt.imshow( (imagesIn[br[i],:,:,:]+1.)/2. )
        plt.axis('off')
    for i in range(10):
        plt.subplot(3,5,i+6)
        plt.imshow( (predictions[i,:,:,:]+1.)/2. )
        plt.axis('off')
    plt.show()